In [21]:
import csv
import psycopg2
from datetime import datetime

In [9]:
# requires postgres db
conn = psycopg2.connect(dbname='postgres', user='postgres', password='postgres')
conn.autocommit = True

In [10]:
cur = conn.cursor()

In [11]:
cur.execute('create user jk')

In [13]:
cur.execute('create database storm_data owner jk')

ProgrammingError: database "storm_data" already exists


In [14]:
conn = psycopg2.connect(dbname='storm_data', user='jk')

In [15]:
cur = conn.cursor()

In [37]:
cur.execute('''drop table if exists storm_data''')
cur.execute('''create table if not exists storm_data (fid int primary key, date timestamp, btid int, name text,
            lat decimal(3,1), long decimal(4,1), wind_kts int, pressure int, cat text, basin text, shape_leng decimal(8,6))''')
conn.commit()

In [20]:
conn.commit()

In [38]:
with open('storm_data.csv', 'r') as f:
    reader = csv.reader(f)
    next(f)
    for row in reader:
        update = [row[0]]
        update.append(datetime(year=int(row[1]), month=int(row[2]), day=int(row[3]), hour=int(row[4][:2]), minute=int(row[4][2:4])))
        update.extend(row[5:])
        #x = update
        #mogrified = cur.mogrify('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s', update).decode('utf-8')
        cur.execute('''insert into storm_data values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', update)
    conn.commit()
    #print(x)

In [40]:
cur.execute('select * from storm_data limit 10')
for a in cur.fetchall():
    print(a)

(2001, datetime.datetime(1957, 8, 8, 18, 0), 63, 'NOTNAMED', Decimal('22.5'), Decimal('-140.0'), 50, 0, 'TS', 'Eastern Pacific', Decimal('1.140175'))
(2002, datetime.datetime(1961, 10, 3, 12, 0), 116, 'PAULINE', Decimal('22.1'), Decimal('-140.2'), 45, 0, 'TS', 'Eastern Pacific', Decimal('1.166190'))
(2003, datetime.datetime(1962, 8, 29, 6, 0), 124, 'C', Decimal('18.0'), Decimal('-140.0'), 45, 0, 'TS', 'Eastern Pacific', Decimal('2.102380'))
(2004, datetime.datetime(1967, 7, 14, 6, 0), 168, 'DENISE', Decimal('16.6'), Decimal('-139.5'), 45, 0, 'TS', 'Eastern Pacific', Decimal('2.121320'))
(2005, datetime.datetime(1972, 8, 16, 12, 0), 251, 'DIANA', Decimal('18.5'), Decimal('-139.8'), 70, 0, 'H1', 'Eastern Pacific', Decimal('1.702939'))
(2006, datetime.datetime(1976, 7, 22, 0, 0), 312, 'DIANA', Decimal('18.6'), Decimal('-139.8'), 30, 0, 'TD', 'Eastern Pacific', Decimal('1.600000'))
(2007, datetime.datetime(1978, 8, 26, 12, 0), 342, 'KRISTY', Decimal('21.4'), Decimal('-140.2'), 45, 0, 'TS',

In [44]:
conn = psycopg2.connect(dbname='storm_data', user='postgres')
cur = conn.cursor()
cur.execute('create group readonly nologin')

In [49]:
conn.rollback()
cur.execute('create group readonly nologin')
cur.execute('''revoke all on storm_data from readonly;
            grant select on storm_data to readonly;
            create user analyst;
            grant readonly to analyst
            ''')
conn.commit()

In [52]:
# trying with additional data but it seems like it requires parsing as the columns don't match up.
with open('storm_data_additional.csv', 'r') as f:
    reader = csv.reader(f)
    next(f)
    for row in reader:
        update = [row[0]]
        update.append(datetime(year=int(row[1]), month=int(row[2]), day=int(row[3]), hour=int(row[4][:2]), minute=int(row[4][2:4])))
        update.extend(row[5:])
        x = update
        #mogrified = cur.mogrify('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s', update).decode('utf-8')
        #cur.execute('''insert into storm_data values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', update)
    #conn.commit()
    print(x)

ValueError: invalid literal for int() with base 10: '1851-06-25 00:00:00'

In [ ]:
# with open('storm_data_additional.csv', 'r') as f:
    reader = csv.reader(f)
    next(f)
    for row in reader:
        print(row)